# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to save regionally averaged sea-ice cover and ice-shelf basal melt as netcdf file
# requires full model output
# note that some paths are hard-coded throughout the script

In [1]:
#!jupyter nbconvert --to script plot_PAPER_carbonate_chemistry_regional_save_netcdf_v2.ipynb

In [2]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd
from tqdm import tqdm

In [3]:
#---
# load mesh info
#---

path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)

ind_no_cavity = np.where(cavities==0)[0]
ind_cavity = np.where(cavities==1)[0]


(72411,)


In [4]:
#----
# FUNCTIONS
#----

def get_intersection(lst1, lst2):  
        # Use of hybrid method 
        temp = set(lst2) 
        lst3 = [value for value in lst1 if value in temp] 
        return lst3
    
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

# get subareas based on lat only
def get_subarea_indices_sectors_lat_only(lat,lon):
    #subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh']
    
    lat_threshold = -60 #-60 # 0=all SH data
        
    ross1 = 150 #160
    ross2 = -130 # -145
    
    eWeddell1 = -10
    
    print ('Subareas: south of '+str(-1*lat_threshold)+'°S...')
        
    shelf_ind = np.zeros(len(lon))
    for i in range(1,len(lon)+1):
        dd = lon[i-1]
        if lat[i-1]<lat_threshold and dd>-52 and dd<eWeddell1: # Filchner-Ronne & Larsen 
            shelf_ind[i-1]=1
        elif lat[i-1]<-74 and dd>-85 and dd<eWeddell1: # Filchner-Ronne & Larsen in the very south
            shelf_ind[i-1]=1
        elif lat[i-1]<lat_threshold and dd>eWeddell1 and dd<30: # eWeddell
            shelf_ind[i-1]=6
        elif lat[i-1]<lat_threshold and dd>30 and dd<90: # Prydz
            shelf_ind[i-1]=2
        elif lat[i-1]<lat_threshold and dd>90 and dd<ross1: # Amery & Australian
            shelf_ind[i-1]=3
        elif lat[i-1]<lat_threshold and dd>ross1:  # Ross Sea 
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd<ross2:   # Ross Sea
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd>ross2 and dd<-69: #dd<-66: # Amundsen & Bellinghausen
            shelf_ind[i-1]=5
        elif lat[i-1]<lat_threshold and dd>-69 and dd<-52: #dd<-66: # Antarctic Peninsula
            wap_lat = -70
            if lat[i-1]>wap_lat:
                shelf_ind[i-1]=7 # pensinsula
            else:
                #shelf_ind[i-1]=7 
                # now find correct separation between Amundsen & Weddell Sea
                if dd>-66:
                    shelf_ind[i-1]=1
                else:
                    shelf_ind[i-1]=5
                  
        # now find correct separation between Amundsen & Weddell Sea
        #if lat[i-1]<=wap_lat:
          
 #       # now find correct separation between Amundsen & Weddell Sea
 #       if lat[i-1]<-66 and dd>-63 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66 and lat[i-1]>-70 and dd>-64.5 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66.8 and lat[i-1]>-70 and dd>-66 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64 and lat[i-1]>-66 and dd>-60.6 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.8 and lat[i-1]>-66 and dd>-62.4 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.7 and lat[i-1]>-65.9 and dd>-62.2 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-63.4 and lat[i-1]>-64.5 and dd>-58 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
            
    return shelf_ind


In [5]:
#---
# load/create masks for subregions (optional: plot map of subregions -> not yet adapted to python3)
#----

shelf_ind = get_subarea_indices_sectors_lat_only(lat,lon)

ind = np.where((topo>2000))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
shelf_ind[ind]=0
#shelf_ind[cavities==1]=0 #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES

weddell = np.zeros_like(shelf_ind)
weddell[shelf_ind==1]=1

eWeddell = np.zeros_like(shelf_ind)
eWeddell[shelf_ind==6]=1

prydz = np.zeros_like(shelf_ind)
prydz[shelf_ind==2]=1

adelie = np.zeros_like(shelf_ind)
adelie[shelf_ind==3]=1

ross = np.zeros_like(shelf_ind)
ross[shelf_ind==4]=1

amundsen = np.zeros_like(shelf_ind)
amundsen[shelf_ind==5]=1

wap = np.zeros_like(shelf_ind)
wap[shelf_ind==7]=1

#----
# get masks for MPAs
#----

path1 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/masks_MPAs/'

f1 = Dataset(path1+'MPA_mask_AntarcticPeninsula_fesom_mesh_COARZE.nc')
mask_WAP_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_1_fesom_mesh_COARZE.nc')
mask_eastAA1_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_2_fesom_mesh_COARZE.nc')
mask_eastAA2_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_3_fesom_mesh_COARZE.nc')
mask_eastAA3_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_Orkney_fesom_mesh_COARZE.nc')
mask_Orkney_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_RossSea_fesom_mesh_COARZE.nc')
mask_RossSea_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_WeddellSea_fesom_mesh_COARZE.nc')
mask_WeddellSea_MPA = f1.variables['mask_fesom'][:]
f1.close()

#------------

subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh',\
             'south of SR4','south of 60S']
subareas = ['south of SR4']
# ADD "south_of_SR4" and "south_of_60S"
# additionally, load masks for "south of SR4" and "south of 60
path_mask_SR4 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_SR4.nc'
path_mask_60S = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_60S.nc'
path_mask_shelfBreak = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_shelf_break.nc'
fileSR4 = Dataset(path_mask_SR4)
file60S = Dataset(path_mask_60S)
file_shelfBreak = Dataset(path_mask_shelfBreak)
maskSR4 = fileSR4.variables['mask'][:]
mask60S = file60S.variables['mask'][:]
mask_shelfBreak = file_shelfBreak.variables['mask'][:]

print (np.min(shelf_ind),np.max(shelf_ind))
latlim = -60
depth_threshold = 2000

#-----
# plot map of where subareas are
#-----

from matplotlib.colors import ListedColormap

# make the color map:
cmap1 = ListedColormap(['white','darkblue','cornflowerblue','firebrick',\
                        'gold','orange','mediumpurple','violet','mediumturquoise'])
cmap1 = ListedColormap(['white',(51./255.,34./255.,136./255.),(136./255.,204./255.,238./255.),\
                  (68./255.,170./255.,153./255.),(17./255.,119./255.,51./255.),\
                  (153./255.,153./255.,51./255.),(221./255.,204./255.,119./255.),\
                   (204./255.,102./255.,119./255.),(136./255.,34./255.,85./255.)])

levels1 = np.arange(0,10,1)

plot_regions = False
if plot_regions:
    dpicnt = 200
    elem_no_nan = mesh.elem[mesh.no_cyclic_elem,:]

    fig8= plt.figure(num=18, figsize=(4,3), dpi=200, facecolor='w', edgecolor='k')
    map = Basemap(projection='spstere',boundinglat=-60,lon_0=-180,resolution='l')
    #map.drawcoastlines()
    map.drawcountries()#
    map.drawmapboundary(fill_color='0.7')
    x, y = map(mesh.x2, mesh.y2)
    shelf_ind[shelf_ind==0]=-999
    mm=plt.tricontourf(x, y, elem_no_nan[::], shelf_ind, levels = levels1, \
                        cmap=cmap1, extend='both') #,vmin=-4, vmax=4)
    cbar = plt.colorbar(orientation='horizontal',fraction=0.075, pad=0.03,shrink=0.75)# ,ticks=cticks)
    #cbar.cmap.set_under('white')
    cbar.remove() 
    #plt.draw()
    lw = 1.2

    topo_plot = np.copy(mesh.topo)
    topo_plot[cavities==1]=-999
    m2=plt.tricontour(x, y, elem_no_nan[::], topo_plot, levels = [2000],linewidths=[lw],colors=['black'])

    save_plot = False
    if save_plot:
        plt.savefig(savepath1+'Map_subareas_for_C_anth_analysis.png',\
                                            dpi = dpicnt, bbox_inches='tight') 
    plt.show()
    #plt.close(fig8)

# Weddell
# eWeddell
# Prydz
# Adelie
# Ross
# Amundsen/Bellinghausen
# Antarctic Pensinsula


Subareas: south of 60°S...
0.0 7.0


In [6]:
#---
# subareas
#---

#weddell[shelf_ind==1]=1
#eWeddell[shelf_ind==6]=1
#prydz[shelf_ind==2]=1
#adelie[shelf_ind==3]=1
#ross[shelf_ind==4]=1
#amundsen[shelf_ind==5]=1
#wap[shelf_ind==7]=1

# for MPAs (outside cavities only for now):
# 1) complete MPA
# 2) shelf only
# 3) open ocean only

#subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
#             'RossSea_MPA','RossSea_MPA_shelf','RossSea_MPA_openOcean',\
#              'WeddellSea_MPA','WeddellSea_MPA_shelf','WeddellSea_MPA_openOcean',\
#              'Orkney_MPA','AntarcticPeninsula_MPA',\
#             'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA',\
#             'East_Antarctica_1_MPA_shelf','East_Antarctica_2_MPA_shelf',\
#              'East_Antarctica_3_MPA_shelf','East_Antarctica_all_MPA_shelf',\
#             'East_Antarctica_1_MPA_openOcean','East_Antarctica_2_MPA_openOcean',\
#              'East_Antarctica_3_MPA_openOcean','East_Antarctica_all_MPA_openOcean']
subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
             'RossSea_MPA','RossSea_MPA_openOcean','RossSea_MPA_shelf',\
              'WeddellSea_MPA','WeddellSea_MPA_openOcean','WeddellSea_MPA_shelf',\
              'Orkney_MPA','AntarcticPeninsula_MPA',\
             'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA',\
             'East_Antarctica_1_MPA_shelf','East_Antarctica_2_MPA_shelf',\
             'East_Antarctica_3_MPA_shelf','East_Antarctica_all_MPA_shelf','East_Antarctica_all_MPA_openOcean',\
             'AntarcticPeninsula_MPA_openOcean','AntarcticPeninsula_MPA_shelf']
print(len(subregions))

#subregions = ['all_shelves']#,'WeddellSea','RossSea','Amundsen_Bellingshausen','WAP']


27


In [10]:
#----
# loop over simulations and years
#----


#indSO = np.where((mesh.y2<-60.0) & (topo<2000.0) & (cavities==0))[0]
#print ('Number of nodes:',indSO.shape)

which_sim = 'simAssp126'
path1 = '/pscratch/sd/c/cnissen/COARZE_ice/'+which_sim+'/' 

if which_sim in ['simAssp126','simAssp245','simAssp370','simCssp245']:
    year_list1 = np.arange(2015,2100+1,1) 
elif which_sim in ['simAssp585','simB','simCssp585']:
    year_list1 = np.arange(1990,2100+1,1) 
    
print(year_list1)

vari_list = ['sea_ice_area','ice_shelf_basal_melt']

sea_ice_area_1         = np.zeros([12,len(year_list1),len(subregions)])
ice_shelf_basal_melt_1 = np.zeros([12,len(year_list1),len(subregions)])

# loop over years
for yy in range(0,len(year_list1)):
    print('year '+str(year_list1[yy])+'...')
                
    #runID = get_runID(which_sim,year_list[yy])

    # open netcdf files
    f1 = Dataset(path1+'sic_fesom_'+str(year_list1[yy])+'0101.nc')
    f2 = Dataset(path1+'wnet_fesom_'+str(year_list1[yy])+'0101.nc')
    f3 = Dataset(path1+'evap_fesom_'+str(year_list1[yy])+'0101.nc') 
    f4 = Dataset(path1+'prlq_fesom_'+str(year_list1[yy])+'0101.nc') 
    f5 = Dataset(path1+'prsn_fesom_'+str(year_list1[yy])+'0101.nc') 
    f6 = Dataset(path1+'runoff_fesom_'+str(year_list1[yy])+'0101.nc') 
    
    for mm in range(0,12): #tqdm(range(0,12)):
        #print ('Month '+str(mm+1)+'...')
        
        for rr in range(0,len(subregions)):
            #print subregions[rr]
            if subregions[rr] in ['all_shelves']:
                ind_regions = np.where(shelf_ind>0)[0]
            elif subregions[rr] in ['WeddellSea']:
                ind_regions = np.where(weddell==1)[0]
            elif subregions[rr] in ['eWeddellSea']:
                ind_regions = np.where(eWeddell==1)[0]
            elif subregions[rr] in ['Prydz']:
                ind_regions = np.where(prydz==1)[0]
            elif subregions[rr] in ['Adelie']:
                ind_regions = np.where(adelie==1)[0]
            elif subregions[rr] in ['RossSea']:
                ind_regions = np.where(ross==1)[0]
            elif subregions[rr] in ['Amundsen_Bellingshausen']:
                ind_regions = np.where(amundsen==1)[0]
            elif subregions[rr] in ['WAP']:
                ind_regions = np.where(wap==1)[0]
            
            elif subregions[rr] in ['RossSea_MPA']:
                ind_regions = np.where((mask_RossSea_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['RossSea_MPA_shelf']:
                ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['RossSea_MPA_openOcean']:
                ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['WeddellSea_MPA']:
                ind_regions = np.where((mask_WeddellSea_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['WeddellSea_MPA_shelf']:
                ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['WeddellSea_MPA_openOcean']:
                ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['AntarcticPeninsula_MPA']:
                ind_regions = np.where((mask_WAP_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['AntarcticPeninsula_MPA_shelf']:
                ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['AntarcticPeninsula_MPA_openOcean']:
                ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['Orkney_MPA']:
                ind_regions = np.where((mask_Orkney_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
                #print(ind_regions.shape)
            elif subregions[rr] in ['Orkney_MPA_shelf']:
                ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['Orkney_MPA_openOcean']:
                ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo>2000))[0]
            
            
            elif subregions[rr] in ['East_Antarctica_1_MPA']:
                ind_regions = np.where((mask_eastAA1_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['East_Antarctica_1_MPA_shelf']:
                ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_1_MPA_openOcean']:
                ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['East_Antarctica_2_MPA']:
                ind_regions = np.where((mask_eastAA2_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['East_Antarctica_2_MPA_shelf']:
                ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_2_MPA_openOcean']:
                ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo>2000))[0]
            elif subregions[rr] in ['East_Antarctica_3_MPA']:
                ind_regions = np.where((mask_eastAA3_MPA==1))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['East_Antarctica_3_MPA_shelf']:
                ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_3_MPA_openOcean']:
                ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo>2000))[0]
            
            elif subregions[rr] in ['East_Antarctica_all_MPA']:
                ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ))[0] #  & (cavities==0) TAKE OUT SAVITY CRITERIO HERE -> TO GET CORRECT BASAL MELT RATES
            elif subregions[rr] in ['East_Antarctica_all_MPA_shelf']:
                ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo<=2000))[0]
            elif subregions[rr] in ['East_Antarctica_all_MPA_openOcean']:
                ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo>2000))[0]
    
            sic    = f1.variables['sic'][mm,:]
            wnet   = f2.variables['wnet'][mm,:]
            evap   = f3.variables['evap'][mm,:]
            prlq   = f4.variables['prlq'][mm,:]
            prsn   = f5.variables['prsn'][mm,:]
            runoff = f6.variables['runoff'][mm,:]
            
            prlq = prlq/1025 # convert rain to m s-1
            prsn = prsn/290 # convert snow to m s-1

            # get sea ice area
            sea_ice_area_1[mm,yy,rr] = np.sum(sic[ind_regions]*area[ind_regions])
            
            # get ice-shelf basal melt
            basal = wnet-prlq-evap-prsn-runoff
            basal[ind_no_cavity] = 0
            basal = basal*86400*365 # convert to m yr-1
            ind_subarea_cav  = get_intersection(ind_regions,ind_cavity)
            #if mm==0:
            #    print(subregions[rr],np.asarray(ind_subarea_cav).shape)
            ice_shelf_basal_melt_1[mm,yy,rr] = np.nansum(basal[ind_subarea_cav]*area[ind_subarea_cav])
            
            #del basal,prlq,prsn,evap,wnet,runoff,sic,ind_subarea_cav
            
    #-----
    # save as netcdf file
    #-----
    
    save_netcdf = True
    # only save every year (takes too much time otherwise)
    if save_netcdf:
        if save_netcdf:
            if save_netcdf:
                savepath = '/pscratch/sd/c/cnissen/COARZE_ice/regional/'
                netcdf_name = 'Sea_ice_area_basal_melt_regional_'+which_sim+'_'+str(year_list1[0])+'_'+str(year_list1[-1])+'_v4.nc'

          #      subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
          #   'RossSea_MPA',\
          #    'WeddellSea_MPA',\
          #    'Orkney_MPA','AntarcticPeninsula_MPA',\
          #   'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA']
                
                if not os.path.exists(savepath+netcdf_name):
                    print('Create file '+savepath+netcdf_name)
                    w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                    w_nc_fid.regions1 = '1=all shelves, 2=Weddell Sea, 3=eWeddell Sea, 4=Prydz, 5=Adelie,'+\
                                ' 6=Ross Sea, 7=Amundsen/Bellingshausen, 8=WAP'
                    w_nc_fid.regions2 = '9=RossSea_MPA, 10=RossSea_MPA_openOcean, 11=RossSea_MPA_shelf',
                    w_nc_fid.regions3 = '12=WeddellSea_MPA, 13=WeddellSea_MPA_openOcean, 14=WeddellSea_MPA_shelf'
                    w_nc_fid.regions4 = '15=Orkney_MPA, 16=AntarcticPeninsula_MPA'
                    w_nc_fid.regions5 = '17=East_Antarctica_1_MPA, 18=East_Antarctica_2_MPA, 19=East_Antarctica_3_MPA, 20=East_Antarctica_all_MPA'
                    w_nc_fid.regions6 = '21=East_Antarctica_1_MPA_shelf, 22=East_Antarctica_2_MPA_shelf, 23=East_Antarctica_3_MPA_shelf'
                    w_nc_fid.regions6 = '24=East_Antarctica_all_MPA_shelf, 25=East_Antarctica_all_MPA_openOcean'
                    w_nc_fid.regions7 = '27=AntarcticPeninsula_MPA_openOcean,28=AntarcticPeninsula_MPA_shelf'
                       
                    w_nc_fid.script    = '/global/homes/c/cnissen/scripts/plot_PAPER_carbonate_chemistry_sea_ice_basal_melt.ipynb'
                    # create dimension & variable
                    w_nc_fid.createDimension('month', 12)  # monthly
                    w_nc_fid.createDimension('year', len(year_list1)) 
                    w_nc_fid.createDimension('subregions', len(subregions)) 
                    w_nc_fid.createDimension('nodes_2d', len(topo)) 
                    w_nc_fid.createVariable('year', 'f4',('year'))
                    w_nc_fid.variables['year'][:] = year_list1

                    w_nc_fid.createVariable('mask_subregions', 'f4',('nodes_2d'))
                    w_nc_fid.variables['mask_subregions'][:] = shelf_ind

                    for vv in range(0,len(vari_list)):
                        w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('month','year','subregions'))
                    w_nc_fid.close()
                else: # file exists, but check if variables exist as well
                    w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                    for vv in range(0,len(vari_list)):
                        try:
                            w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('month','year','subregions'))
                        except: 
                            if yy==0:
                                print (vari_list[vv]+' exists in file')
                            pass
                    w_nc_fid.close()
                    

                for vv in range(0,len(vari_list)):
                    w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                    if vari_list[vv] in ['sea_ice_area']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = sea_ice_area_1[:,yy,:]
                    elif vari_list[vv] in ['ice_shelf_basal_melt']:
                        w_nc_fid.variables[vari_list[vv]][:,yy,:] = ice_shelf_basal_melt_1[:,yy,:]

                    w_nc_fid.close()  
                del netcdf_name
                

    # close netcdf files
    f1.close()
    f2.close()
    f3.close()
    f4.close()
    f5.close()
    f6.close()

print ('done')


[2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028
 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042
 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056
 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070
 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084
 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098
 2099 2100]
year 2015...


/tmp/ipykernel_1336741/1429824695.py:133: UserWarning: Warning: converting a masked element to nan.
  ice_shelf_basal_melt_1[mm,yy,rr] = np.nansum(basal[ind_subarea_cav]*area[ind_subarea_cav])


Create file /pscratch/sd/c/cnissen/COARZE_ice/regional/Sea_ice_area_basal_melt_regional_simAssp126_2015_2100_v4.nc
year 2016...
year 2017...
year 2018...
year 2019...
year 2020...
year 2021...
year 2022...
year 2023...
year 2024...
year 2025...
year 2026...
year 2027...
year 2028...
year 2029...
year 2030...
year 2031...
year 2032...
year 2033...
year 2034...
year 2035...
year 2036...
year 2037...
year 2038...
year 2039...
year 2040...
year 2041...
year 2042...
year 2043...
year 2044...
year 2045...
year 2046...
year 2047...
year 2048...
year 2049...
year 2050...
year 2051...
year 2052...
year 2053...
year 2054...
year 2055...
year 2056...
year 2057...
year 2058...
year 2059...
year 2060...
year 2061...
year 2062...
year 2063...
year 2064...
year 2065...
year 2066...
year 2067...
year 2068...
year 2069...
year 2070...
year 2071...
year 2072...
year 2073...
year 2074...
year 2075...
year 2076...
year 2077...
year 2078...
year 2079...
year 2080...
year 2081...
year 2082...
year 2083...
y